In [1]:
!pip -qq install torchutils mplcyberpunk seaborn

In [2]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T
from torchvision.models import resnet18, ResNet18_Weights

# Для чтения изображений с диска
from torchvision import io # input/output
import torchutils as tu
import json
import mplcyberpunk
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import mlflow      
import mlflow.pytorch  


torch.manual_seed(42)
plt.style.use('cyberpunk')

/home/lesia/DS/projects/phase 2 week 1/nn_project_stars_25/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cpu


In [5]:
train_dir = "../data/dataset2-master/dataset2-master/images/TRAIN"
test_dir = "../data/dataset2-master/dataset2-master/images/TEST"

In [6]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

print(train_dataset.classes)

['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']


In [11]:
class_names = train_dataset.classes

In [12]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [13]:
n_classes = len(train_dataset.classes)

In [14]:
model = resnet18(weights=ResNet18_Weights.DEFAULT).to(DEVICE)
fake_batch = torch.randn(1, 3, 224, 224)
tu.get_model_summary(model, fake_batch)

Layer                                       Kernel             Output          Params           FLOPs
0_conv1                                   [3, 64, 7, 7]   [1, 64, 112, 112]       9,408   118,013,952
1_bn1                                              [64]   [1, 64, 112, 112]         128     3,211,264
2_relu                                                -   [1, 64, 112, 112]           0             0
3_maxpool                                             -     [1, 64, 56, 56]           0             0
4_layer1.0.Conv2d_conv1                  [64, 64, 3, 3]     [1, 64, 56, 56]      36,864   115,605,504
5_layer1.0.BatchNorm2d_bn1                         [64]     [1, 64, 56, 56]         128       802,816
6_layer1.0.ReLU_relu                                  -     [1, 64, 56, 56]           0             0
7_layer1.0.Conv2d_conv2                  [64, 64, 3, 3]     [1, 64, 56, 56]      36,864   115,605,504
8_layer1.0.BatchNorm2d_bn2                         [64]     [1, 64, 56, 56]       

In [15]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [17]:
for param in model.parameters():
    param.requires_grad = False

In [18]:
model.fc = nn.Linear(model.fc.in_features, n_classes)
model = model.to(DEVICE)

In [19]:
for param in model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


In [20]:
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

In [21]:
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [22]:
def fit(
    model: torch.nn.Module,
    n_epochs: int,
    optimizer: torch.optim.Optimizer,
    train_loader: DataLoader,
    valid_loader: DataLoader
):

    train_losses = []
    valid_losses = []
    train_acc = []
    valid_acc = []

    for epoch in range(n_epochs):

        # ===== TRAIN =====
        model.train()
        batch_loss = []
        batch_accs = []

        for images, labels in train_loader:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            preds = model(images)
            loss = criterion(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batch_loss.append(loss.item())

            acc = (preds.argmax(dim=1) == labels).float().mean().item()
            batch_accs.append(acc)

        train_losses.append(np.mean(batch_loss))
        train_acc.append(np.mean(batch_accs))

        # ===== VALID =====
        model.eval()
        batch_loss = []
        batch_accs = []

        with torch.no_grad():
            for images, labels in valid_loader:
                images = images.to(DEVICE)
                labels = labels.to(DEVICE)

                preds = model(images)
                loss = criterion(preds, labels)

                batch_loss.append(loss.item())

                acc = (preds.argmax(dim=1) == labels).float().mean().item()
                batch_accs.append(acc)

        valid_losses.append(np.mean(batch_loss))
        valid_acc.append(np.mean(batch_accs))

        print(f"Epoch {epoch+1}/{n_epochs}")
        print(f"Train loss: {train_losses[-1]:.4f} | Train acc: {train_acc[-1]:.4f}")
        print(f"Valid loss: {valid_losses[-1]:.4f} | Valid acc: {valid_acc[-1]:.4f}")
        print("-" * 40)

    return train_losses, valid_losses, train_acc, valid_acc


In [23]:
train_losses, valid_losses, train_acc, valid_acc = fit(
    model,
    n_epochs=5,
    optimizer=optimizer,
    train_loader=train_loader,
    valid_loader=valid_loader
)

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 4))

# ---- LOSS ----
ax[0].plot(train_losses, label='train loss')
ax[0].plot(valid_losses, label='valid loss')
ax[0].set_title(f'Loss per epoch ({len(train_losses)} epochs)')
ax[0].grid(True)
ax[0].legend()

# ---- ACC ----
ax[1].plot(train_acc, label='train acc')
ax[1].plot(valid_acc, label='valid acc')
ax[1].set_title(f'Accuracy per epoch ({len(train_acc)} epochs)')
ax[1].grid(True)
ax[1].set_ylim(0, 1)
ax[1].legend()

plt.show()

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix

f1 = f1_score(y_true, y_pred, average="macro")
print("F1:", f1)

cm = confusion_matrix(y_true, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt="d")
plt.show()


In [ ]:
torch.save(model.state_dict(), "models/blood_resnet18.pth")